In [322]:
'''The Jupyter notebook is a web-based notebook environment for interactive computing. 
The Jupyter Notebook is an open source web application that you can use to create and share documents that 
contain live code, equations, visualizations, and text.
'''
print(f"** es-search-guard")
'''  Search Guard Is An Open Source Security Plugin For Elasticsearch  '''

** es-search-guard


'  Search Guard Is An Open Source Security Plugin For Elasticsearch  '

In [323]:
#pip install elasticsearch==7.13.0

In [324]:
from elasticsearch import Elasticsearch
import os
import json
import sys
import pandas as pd
import datetime
from datetime import datetime
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")

In [325]:
sys.path.append(r"C://Users/euiyoung.hwang/Git_Workspace/ELK-Stack-Upgrade/jupyter-workflow/")

In [326]:
import pytest
import ipytest

In [327]:
''' pip install python-dotenv'''
load_dotenv() # will search for .env file in local folder and load variables 

True

* When security features are enabled, depending on the realms you’ve configured, you must attach your user credentials to the requests sent to Elasticsearch. For example, when using realms that support usernames and passwords you can simply attach basic auth header to the requests. (A realm is a security policy domain defined for a web or application server.) : https://www.elastic.co/guide/en/elasticsearch/reference/current/setting-up-authentication.html
* In order to search or index with an ES cluster based on x-pack, you need to add a user and grant permission. I created an admin account, a wmx/omx user, etc in Dev ES cluster with Search Guard as X-pack.
* Basic Authentication is a method for an HTTP user agent (e.g., a web browser) to provide a username and password when making a request. You can send the authorization header when making requests and accessing to ES Cluster based on Search-Guard as X-pack. The value is passed to the authorization key value in the http header in the format below.
* Basic Auth : {'Content-type': 'application/json', 'Authorization' : 'Basic base64.encode(id:password), 'Connection': 'close'}

In [328]:
''' Basic authentication is a very simple authentication scheme that is built into the HTTP protocol. The client sends HTTP requests with the Authorization header that contains the Basic word '''
def get_headers():
    ''' Elasticsearch Header '''
    return {'Content-type': 'application/json', 'Authorization' : '{}'.format(os.getenv('JUPYTER_BASIC_AUTH')), 'Connection': 'close'}

### User Authorzation Check via ES cluster with no X-pack
* ES Version : ES v.5.6.4 (Dev ES Cluster)
* X-pack : Not Installed

In [329]:
# display(os.getenv('ES_DEV_V8_HOST'))

In [330]:
''' Create ES Instance once user credential was added into headers. If it is http-based, even if the header value is sent, it is ignored. '''
''' https://www.debugbear.com/basic-auth-header-generator '''
''' Call API to DEV ES Cluster '''
es = Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), headers=get_headers(), timeout=5,  verify_certs=False)

In [331]:
print(es.cat.health())

1740178618 22:56:58 supplychain-logging-qa13-es8 green 3 3 293 251 0 0 0 0 0 - 100.0%



In [332]:
print(json.dumps(es.info(), indent=2))

{
  "name": "supplychain-logging-qa13-1",
  "cluster_name": "supplychain-logging-qa13-es8",
  "cluster_uuid": "iiQ1fnsxSz2c0VDmhWcctg",
  "version": {
    "number": "8.17.0",
    "build_flavor": "default",
    "build_type": "tar",
    "build_hash": "2b6a7fed44faa321997703718f07ee0420804b41",
    "build_date": "2024-12-11T12:08:05.663969764Z",
    "build_snapshot": false,
    "lucene_version": "9.12.0",
    "minimum_wire_compatibility_version": "7.17.0",
    "minimum_index_compatibility_version": "7.0.0"
  },
  "tagline": "You Know, for Search"
}


In [333]:
print(f"es version : {es.info()['version']['number']}")

es version : 8.17.0


In [334]:
df = pd.DataFrame.from_dict([es.cluster.health()])
print(json.dumps(es.cluster.health(), indent=2))
display(df)

{
  "cluster_name": "supplychain-logging-qa13-es8",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 3,
  "number_of_data_nodes": 3,
  "active_primary_shards": 251,
  "active_shards": 293,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "unassigned_primary_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


,cluster_name,status,timed_out,number_of_nodes,number_of_data_nodes,active_primary_shards,active_shards,relocating_shards,initializing_shards,unassigned_shards,unassigned_primary_shards,delayed_unassigned_shards,number_of_pending_tasks,number_of_in_flight_fetch,task_max_waiting_in_queue_millis,active_shards_percent_as_number
0,supplychain-logging-qa13-es8,green,False,3,3,251,293,0,0,0,0,0,0,0,0,100.0


### User Authentication Check via ES cluster with Search Guard
* ES Version : ES v.8.12 via https protocal
* X-pack : Search Guard for ES v.8.12
* Let's check the authentication on the Dev ES Cluster with Search Guard installed.

In [335]:
#pip install requests

In [336]:
import base64
import requests

In [337]:
# display(os.getenv('ES_DEV_V8_HOST'))

In [338]:
''' Search Guard Health '''
resp = requests.get(url=f"{os.getenv('ES_DEV_V8_HOST')}/_searchguard/health", headers=None, verify=False, timeout=600)
print(resp)
# print(json.dumps(resp.json(), indent=2))
display(pd.DataFrame([resp.json()]))

<Response [200]>


,message,mode,status
0,None,strict,UP


In [339]:
''' https://www.debugbear.com/basic-auth-header-generator '''
''' CASE#1 - Check when making request with https es host url and headers don't have basic auth info '''
es_search_guard = Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), headers=None, timeout=5,  verify_certs=False)

In [340]:
''' Getting error 'AuthenticationException(401, 'Unauthorized') '''
try:
    print(es_search_guard.cat.health())
except Exception as e:
    print(e)

AuthenticationException(401, 'Unauthorized')


In [341]:
''' test base64 encode '''
def base64_encode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    encoded = '{}'.format(base64.b64encode(id_pass.encode('utf-8')).decode())
    # print(encoded)
    return encoded

In [342]:
''' test base64 decode '''
def base64_decode_for_search_guard(id_pass):
    ''' format -> <id>:<password> '''
    return base64.b64decode(id_pass).decode('utf-8')

In [343]:
''' Generate base64 encode for id:pasword when making request to ES cluster with X-pack '''
encoded = base64_encode_for_search_guard('test:1')
print(f"encoded test : {encoded}")
decoded = base64_decode_for_search_guard(encoded)
print(f"decoded test : {decoded}")

encoded test : dGVzdDox
decoded test : test:1


In [344]:
''' https://www.debugbear.com/basic-auth-header-generator '''
''' CASE#2 - Check when making request with https es host url and headers have wrong basic auth info that ES search guard doesn't have test user'''
es_search_guard = Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), 
                                headers={
                                    'Content-type': 'application/json', 
                                    'Authorization' : 'Basic {}'.format(base64_encode_for_search_guard('es_admin:1')), 
                                    'Connection': 'close'
                                }, 
                                timeout=5,  
                                verify_certs=False)

In [345]:
''' Getting error 'AuthenticationException(401, 'Unauthorized') '''
''' There's a problem with 401 Unauthorized, the HTTP status code for authentication errors. And that’s just it: it’s for authentication, not authorization 
The user needs to provide valid authentication credentials, such as a username and password or an access token, and include them in the request headers.
'''
try:
    print(es_search_guard.cat.health())
except Exception as e:
    print(e)

1740178620 22:57:00 supplychain-logging-qa13-es8 green 3 3 293 251 0 0 0 0 0 - 100.0%



In [346]:
# print(get_headers())

In [347]:
''' https://www.debugbear.com/basic-auth-header-generator '''
''' CASE#3 - Check when making request with https es host url and headers have correct basic auth info. Before testing. we need to add a new user with right permission in ES v.8 Cluster '''
''' Call to Dev ES Cluster based on ES v.8.12 using admin account with Basic Auth and Search-Guard as X-pack '''
es_search_guard = Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), headers=get_headers(), timeout=5,  verify_certs=False)

In [348]:
''' Getting the response correctly '''
try:
    print(es_search_guard.cat.health())
    # print(json.dumps(es_search_guard.info(), indent=2))
    df = pd.DataFrame.from_dict([es_search_guard.cluster.health()])
    print(json.dumps(es_search_guard.cluster.health(), indent=2))
    display(df)
except Exception as e:
    print(e)

1740178620 22:57:00 supplychain-logging-qa13-es8 green 3 3 293 251 0 0 0 0 0 - 100.0%

{
  "cluster_name": "supplychain-logging-qa13-es8",
  "status": "green",
  "timed_out": false,
  "number_of_nodes": 3,
  "number_of_data_nodes": 3,
  "active_primary_shards": 251,
  "active_shards": 293,
  "relocating_shards": 0,
  "initializing_shards": 0,
  "unassigned_shards": 0,
  "unassigned_primary_shards": 0,
  "delayed_unassigned_shards": 0,
  "number_of_pending_tasks": 0,
  "number_of_in_flight_fetch": 0,
  "task_max_waiting_in_queue_millis": 0,
  "active_shards_percent_as_number": 100.0
}


,cluster_name,status,timed_out,number_of_nodes,number_of_data_nodes,active_primary_shards,active_shards,relocating_shards,initializing_shards,unassigned_shards,unassigned_primary_shards,delayed_unassigned_shards,number_of_pending_tasks,number_of_in_flight_fetch,task_max_waiting_in_queue_millis,active_shards_percent_as_number
0,supplychain-logging-qa13-es8,green,False,3,3,251,293,0,0,0,0,0,0,0,0,100.0


In [349]:
print(f"es version : {es_search_guard.info()['version']['number']}")

es version : 8.17.0


### Role based User Permission check
* I've already created a different users with different permssion for indices in Dev ES v.8.12 with Search Guard.
* Let's test searching and indexing with the user-specific permissions below.
* admin user (all permission)
* omusers (only access om* indices with cluster api)
* wxusers (only access wx* indices with cluster api)

In [350]:
''' Search Guard retrieves the user’s credentials from the request. '''
''' Search Guard authorizes the user by retrieving a list of the user’s roles from the configured authorization backend '''
''' https://subin-0320.tistory.com/174 '''
print(f"** Role based User Permission check")

** Role based User Permission check


In [351]:
pd.set_option('max_colwidth', 800)

In [352]:
# display(pd.DataFrame([{"Dev ES" : es_search_guard}]))

In [353]:
print(f"** Validate search api with admin account")

** Validate search api with admin account


In [354]:
''' write doc with admin user using es client instance'''
doc = {
    "author": "kimchy",
    "text": "Elasticsearch: cool. bonsai cool.",
    "timestamp": datetime.now(),
}

idx_lists = ['wx_test', 'om_test']

for CR_ADMIN_INDEX in idx_lists:
    if es_search_guard.indices.exists(index=CR_ADMIN_INDEX):
        es_search_guard.indices.delete(index=CR_ADMIN_INDEX)
        print(f"Index was deleted with admin account")
    resp = es_search_guard.index(index=CR_ADMIN_INDEX, id=12, body=doc)
    print("Index was {} with admin account".format(json.dumps(resp["result"], indent=2)))

Index was deleted with admin account
Index was "created" with admin account
Index was deleted with admin account
Index was "created" with admin account


In [355]:
''' search by id for wx_text index with admin user using es client instance'''
resp = es_search_guard.get(index=CR_ADMIN_INDEX, id=12)
print(json.dumps(resp, indent=2))

{
  "_index": "om_test",
  "_id": "12",
  "_version": 1,
  "_seq_no": 0,
  "_primary_term": 1,
  "found": true,
  "_source": {
    "author": "kimchy",
    "text": "Elasticsearch: cool. bonsai cool.",
    "timestamp": "2025-02-21T16:57:01.493656"
  }
}


In [356]:
query={"query": {"match_all": {}}}
resp = es_search_guard.search(index="om_test", body=query)
print(json.dumps(resp['hits'], indent=2))

{
  "total": {
    "value": 0,
    "relation": "eq"
  },
  "max_score": null,
  "hits": []
}


In [357]:
print(f"** Validate search api with omusers account")

** Validate search api with omusers account


In [358]:
es_om_user_es_instance= Elasticsearch(hosts=os.getenv('ES_DEV_V8_HOST'), 
                                headers={
                                    'Content-type': 'application/json', 
                                    'Authorization' : 'Basic {}'.format(base64_encode_for_search_guard('omusers:1')), 
                                    'Connection': 'close'
                                }, 
                                timeout=5,  
                                verify_certs=False)

In [359]:
''' It can be accessed and got the result with omusers account '''
query={"query": {"match_all": {}}}
try:
    resp = es_om_user_es_instance.search(index="om_test", body=query)
    print(json.dumps(resp['hits'], indent=2))
except Exception as e:
    print(f"Validate the permission with om* indices for searching : {e}")

{
  "total": {
    "value": 0,
    "relation": "eq"
  },
  "max_score": null,
  "hits": []
}


In [360]:
''' AuthorizationException(403, 'security_exception', 'Insufficient permissions') when omusers does try to access wx* index '''
''' omusers doesn't have the permission to access for wx* indices '''
''' Only indices starting with om are searchable. '''
query={"query": {"match_all": {}}}
try:
    resp = es_om_user_es_instance.search(index="wx_test", body=query)
    print(json.dumps(resp['hits'], indent=2))
except Exception as e:
    print(f"Validate the permission with wx* indices for searching : {e}")

Validate the permission with wx* indices for searching : AuthorizationException(403, 'security_exception', 'Insufficient permissions')


In [361]:
''' omusers don't have the permission to create index and write docuements '''
doc = {
    "author": "kimchy",
    "text": "Elasticsearch: cool. bonsai cool.",
    "timestamp": datetime.now(),
}
try:
    resp = es_om_user_es_instance.index(index="wx-test-index", id=1, body=doc)
    print(json.dumps(resp["result"], indent=2))
except Exception as e:
    print(f"Validate the permission for indexing : {e}")

Validate the permission for indexing : AuthorizationException(403, 'security_exception', 'Insufficient permissions')


In [362]:
print(f"** Role based User Permission check using requests library")

** Role based User Permission check using requests library


In [363]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [364]:
''' https://restfulapi.net/http-status-codes/ '''
http_status_code = {
    200 : 'Indicates that the request has succeeded.',
    201 : 'Indicates that the request has succeeded and a new resource has been created as a result.',
    400 : 'The server could not understand the request due to incorrect syntax. The client should NOT repeat the request without modifications.',
    401 : 'Unauthorized',
    403 : 'Unauthorized request. The client does not have access rights to the content. ',
    500 : 'The server encountered an unexpected condition that prevented it from fulfilling the request.'
}

In [365]:
''' Verify user's permission using the API such as create index or delete index or search'''
''' User Permission with a different role for ES v.8.12.2 with x-pack security based on Searcch Guard '''
api_list = ['_cat/nodes?format=json', '_cat/aliases?format=json', '_cat/nodes?format=json', 
            'wx_test/_search', 'om_test/_search', 'logstash-2024.08.21/_search'
           ]
user_list = os.getenv('USER_LIST').split(",")
for user in user_list:
    ''' Add header for each user '''
    header = {'Content-type': 'application/json', 'Authorization' : 'Basic {}'.format(base64_encode_for_search_guard(user)), 'Connection': 'close'}
    print('-'*50)

    IDX = 'logstash-2025-01-test'
    ''' Verify write permission'''
    body = [
                {"index" : { "_index" : IDX, "_id" : "1"}},
                {"ADDTS" : "03/09/2023 02:06:34.739993" },
                {"index" : { "_index" : IDX, "_id" : "2"}},
                {"ADDTS" : "03/09/2023 02:06:34.739993" }
    ]

    payload = '\n'.join([json.dumps(line) for line in body]) + '\n'
    print(f"User : {user.split(':')[0]}")
    resp = requests.post(url="{}/_bulk".format(os.getenv('ES_DEV_V8_HOST')), headers=header, data=payload, verify=False, timeout=5)
    ''' check first records if user has permission to write'''
    status = resp.json()["items"][0]["index"]["status"]
    if "errors" in resp.json() and resp.json()["errors"] == True:
        # print(f"POST [{status, http_status_code.get(status, '')}] User ['{user.split(':')[0]}'] - CREATE INDEX Request : {resp.json()}")
        print(f"POST [{status, {http_status_code.get(status, '')}}] User ['{user.split(':')[0]}'] - CREATE INDEX Request")
    else:
        # print(f"POST [{resp.status_code, http_status_code.get(resp.status_code, '')}] User ['{user.split(':')[0]}'] - CREATE INDEX Request : {resp.json()}")
        print(f"POST [{resp.status_code}, {http_status_code.get(resp.status_code, '')}] User ['{user.split(':')[0]}'] - CREATE INDEX Request")

    ''' Verif delete permission '''
    resp = requests.delete(url="{}/{}".format(os.getenv('ES_DEV_V8_HOST'), IDX), headers=header, verify=False, timeout=600)
    print(f"DELETE [{resp.status_code}, {http_status_code.get(resp.status_code, '')}] User ['{user.split(':')[0]}'] - DELETE INDEX Request")

    ''' create again for curator account '''
    resp = requests.post(url="{}/_bulk".format(os.getenv('ES_DEV_V8_HOST')), headers=header, data=payload, verify=False, timeout=5)

    ''' _search '''
    for api in api_list:
        ''' Verify search permission '''
        resp = requests.get(url="{}/{}".format(os.getenv('ES_DEV_V8_HOST'), api), headers=header, verify=False, timeout=5)
        color_print = color.GREEN if resp.status_code == 200 else color.RED
        # print(f"{color.BOLD}{color_print} GET {[resp.status_code , http_status_code.get(resp.status_code, '')]} User [{user.split(':')[0]}], Urls [{api}]{color.END}")
        print(f"GET {[resp.status_code , http_status_code.get(resp.status_code, '')]} User [{user.split(':')[0]}], Urls [{api}]")

--------------------------------------------------
User : es_admin
POST [200, Indicates that the request has succeeded.] User ['es_admin'] - CREATE INDEX Request
DELETE [200, Indicates that the request has succeeded.] User ['es_admin'] - DELETE INDEX Request
GET [200, 'Indicates that the request has succeeded.'] User [es_admin], Urls [_cat/nodes?format=json]
GET [200, 'Indicates that the request has succeeded.'] User [es_admin], Urls [_cat/aliases?format=json]
GET [200, 'Indicates that the request has succeeded.'] User [es_admin], Urls [_cat/nodes?format=json]
GET [200, 'Indicates that the request has succeeded.'] User [es_admin], Urls [wx_test/_search]
GET [200, 'Indicates that the request has succeeded.'] User [es_admin], Urls [om_test/_search]
GET [404, ''] User [es_admin], Urls [logstash-2024.08.21/_search]
--------------------------------------------------
User : es_logstash
POST [200, Indicates that the request has succeeded.] User ['es_logstash'] - CREATE INDEX Request
DELETE [2

### IPytest
* ipytest allows you to run Pytest in Jupyter notebooks.
* The pytest framework makes it easy to write small tests. It can be used to write various types of software tests, including unit tests and others

In [366]:
import pytest
''' https://github.com/chmp/ipytest?tab=readme-ov-file '''
''' https://github.com/chmp/ipytest/blob/main/Example.ipynb '''
''' https://docs.search-guard.com/latest/offline-tls-tool '''
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

In [367]:
!pip install pytest-cov

In [368]:
''' To begin, you need to install pytest and ipytest, a tool designed to run pytest tests directly in Jupyter. Execute the following in a Jupyter cell '''
ipytest.autoconfig(coverage=True)

In [369]:
''' ******* Test code ********* '''
# @pytest.mark.skip(reason="no way of currently testing this")
# def test_pytest_skip_func():
#     assert 42 == 42

In [370]:
def test_base64_func():
    # resp = es_search_guard.ping()
    # assert resp == True
    expected_encoded ='dGVzdDox'
    expected_decoded = 'test:1'
    encoded = base64_encode_for_search_guard(expected_decoded)
    assert "Basic {}".format(encoded) == 'Basic {}'.format(expected_encoded)
    assert base64_decode_for_search_guard(expected_encoded) == expected_decoded

In [371]:
def test_installed_running_search_guard_es_cluster_func():
    ''' Check if search guard was installed in ES cluster via _searchguard api '''
    resp = requests.get(url=f"{os.getenv('ES_DEV_V8_HOST')}/_searchguard/health", headers=None, verify=False, timeout=600)
    assert resp.status_code == 200
    assert resp.json() == {
          "message": None,
          "mode": "strict",
          "status": "UP"
    }

In [372]:
@pytest.mark.skip(reason="no way of currently testing this")
def test_sg_role_validation_admin_account_func():
    ''' sg_role for admin account '''
    mock_es_admin_client = es_search_guard
    assert mock_es_admin_client is not None
    resp = mock_es_admin_client.get(index="wx_test", id=12)
    assert resp is not None
    assert resp == {
      "_index": "wx_test",
      "_id": "12",
      "_version": 1,
      "_seq_no": 0,
      "_primary_term": 1,
      "found": True,
      "_source": {
        "ADDTS": "03/09/2023 02:06:34.739993"
      }
    }

    query={"query": {"match_all": {}}}
    resp = mock_es_admin_client.search(index="om_test", body=query)
    assert resp is not None
    assert resp['hits'] == {
      "total": {
        "value": 1,
        "relation": "eq"
      },
      "max_score": 1.0,
      "hits": [
        {
          "_index": "om_test",
          "_id": "12",
          "_score": 1.0,
          "_source": {
            "ADDTS": "03/09/2023 02:06:34.739993"
          }
        }
      ]
    }
    assert len(resp['hits']) > 0

In [373]:
''' Execute the tests using ipytest.run(). You can pass command-line arguments to control test behavior: '''
''' The pytest framework makes it easy to write small, readable tests, and can scale to support complex functional testing for applications and libraries. '''
ipytest.run('-vv')
# ipytest.run('-qq')

======================================= test session starts =======================================
platform win32 -- Python 3.11.7, pytest-8.2.2, pluggy-1.5.0 -- C:\Users\euiyoung.hwang\Git_Workspace\ELK_Stack_Upgrade\.venv\Scripts\python.exe
cachedir: .pytest_cache
rootdir: C:\Users\euiyoung.hwang\Git_Workspace\ELK_Stack_Upgrade
configfile: pyproject.toml
plugins: anyio-4.8.0, cov-6.0.0
collected 3 items. 

PASSED                               [ 33%]
PASSED [ 66%]49a.py::test_installed_running_search_guard_es_cluster_func 
SKIPPED    [100%]py::test_sg_role_validation_admin_account_func 

---------- coverage: platform win32, python 3.11.7-final-0 -----------
Name                                                                       Stmts   Miss  Cover
----------------------------------------------------------------------------------------------
C:\Users\euiyoung.hwang\AppData\Local\Temp\ipykernel_30576\1095938805.py       6      1    83%
C:\Users\euiyoung.hwang\AppData\Local\Temp\ipyk

<ExitCode.OK: 0>